<a href="https://colab.research.google.com/github/mahimaboppi/chatbox/blob/main/Kirana_Inventory_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime

# Inventory and Sales Log DataFrames
inventory = pd.DataFrame(columns=['ProductID', 'Name', 'Category', 'Price', 'Quantity'])
sales_log = pd.DataFrame(columns=['Date', 'ProductID', 'Name', 'Quantity', 'TotalPrice'])

# Add a new product
def add_product(product_id, name, category, price, quantity):
    global inventory
    if product_id in inventory['ProductID'].values:
        print("❌ Product ID already exists.")
        return
    new_product = pd.DataFrame([{
        'ProductID': product_id,
        'Name': name,
        'Category': category,
        'Price': price,
        'Quantity': quantity
    }])
    inventory = pd.concat([inventory, new_product], ignore_index=True)
    print("✅ Product added!")

# Update product price
def update_price(product_id, new_price):
    global inventory
    if product_id not in inventory['ProductID'].values:
        print("❌ Product ID not found.")
        return
    inventory.loc[inventory['ProductID'] == product_id, 'Price'] = new_price
    print(f"💰 Price updated to ₹{new_price}")

# View inventory
def view_inventory():
    if inventory.empty:
        print("📭 Inventory is empty.")
    else:
        print(inventory.to_string(index=False))

# Generate a bill
def generate_bill(purchases):
    global inventory, sales_log
    bill_items = []
    total_amount = 0
    errors = []

    for pid, qty in purchases:
        product = inventory[inventory['ProductID'] == pid]
        if product.empty:
            errors.append(f"❌ Product ID {pid} not found.")
            continue
        available_qty = product['Quantity'].values[0]
        name = product['Name'].values[0]
        price = product['Price'].values[0]

        if qty > available_qty:
            errors.append(f"⚠️ Not enough stock for '{name}'")
            continue

        inventory.loc[inventory['ProductID'] == pid, 'Quantity'] -= qty
        item_total = price * qty
        total_amount += item_total
        bill_items.append((pid, name, price, qty, item_total))

        # Log sale
        sales_log = pd.concat([sales_log, pd.DataFrame([{
            'Date': datetime.now().strftime('%Y-%m-%d'),
            'ProductID': pid,
            'Name': name,
            'Quantity': qty,
            'TotalPrice': item_total
        }])], ignore_index=True)

    # Print bill
    print("🧾 KIRANA STORE BILL")
    print(f"🕒 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 40)
    print(f"{'Item':<10} {'Price':<6} {'Qty':<4} {'Total':<6}")
    print("-" * 40)
    for _, name, price, qty, total in bill_items:
        print(f"{name:<10} ₹{price:<6} {qty:<4} ₹{total:<6}")
    print("-" * 40)
    print(f"{'TOTAL':<22} ₹{total_amount}")
    print("-" * 40)

    if errors:
        print("\n⚠️ Issues:")
        for err in errors:
            print(err)

# Generate sales report
def generate_sales_report(period='daily'):
    global sales_log
    if sales_log.empty:
        print("📭 No sales data.")
        return

    if period == 'daily':
        report = sales_log.groupby('Date').agg({'Quantity': 'sum', 'TotalPrice': 'sum'}).reset_index()
        print("📊 Daily Sales Report:")
        print(report.to_string(index=False))
    elif period == 'weekly':
        sales_log['Week'] = pd.to_datetime(sales_log['Date']).dt.isocalendar().week
        report = sales_log.groupby('Week').agg({'Quantity': 'sum', 'TotalPrice': 'sum'}).reset_index()
        print("📊 Weekly Sales Report:")
        print(report.to_string(index=False))
    else:
        print("❌ Invalid period. Use 'daily' or 'weekly'.")

# Save inventory and sales
def save_all():
    inventory.to_csv("inventory.csv", index=False)
    sales_log.to_csv("sales.csv", index=False)
    print("💾 Data saved.")

# Load inventory and sales
def load_all():
    global inventory, sales_log
    try:
        inventory = pd.read_csv("inventory.csv")
        sales_log = pd.read_csv("sales.csv")
        print("📂 Data loaded.")
    except Exception as e:
        print("❌ Error loading data:", e)


In [2]:
# Add products
add_product("P001", "Maggie", "Snacks", 15, 100)
add_product("P002", "Soap", "Toiletries", 30, 50)

# View inventory
view_inventory()

# Generate a bill
generate_bill([("P001", 5), ("P002", 2)])

# Sales report
generate_sales_report("daily")

# Save and load
save_all()
load_all()


✅ Product added!
✅ Product added!
ProductID   Name   Category Price Quantity
     P001 Maggie     Snacks    15      100
     P002   Soap Toiletries    30       50
🧾 KIRANA STORE BILL
🕒 2025-04-05 10:11:59
----------------------------------------
Item       Price  Qty  Total 
----------------------------------------
Maggie     ₹15     5    ₹75    
Soap       ₹30     2    ₹60    
----------------------------------------
TOTAL                  ₹135
----------------------------------------
📊 Daily Sales Report:
      Date Quantity TotalPrice
2025-04-05        7        135
💾 Data saved.
📂 Data loaded.


In [3]:
def save_all():
    inventory.to_csv("inventory.csv", index=False)
    sales_log.to_csv("sales.csv", index=False)
    print("💾 Data saved to inventory.csv and sales.csv")


In [4]:
def load_all():
    global inventory, sales_log
    try:
        inventory = pd.read_csv("inventory.csv")
        sales_log = pd.read_csv("sales.csv")
        print("📂 Data loaded successfully.")
    except FileNotFoundError:
        print("⚠️ Files not found. Make sure to save first or check the file names.")
    except Exception as e:
        print(f"❌ Error loading data: {e}")


In [5]:
# After adding products or making sales
save_all()

# When reopening the notebook next time
load_all()


💾 Data saved to inventory.csv and sales.csv
📂 Data loaded successfully.
